# Serial implementaiton of the 1D heat equation introducing partitioning 

* In this code partitions are introduced such that we can controlt the grain size (amount of work per future) to gain better scalabilty 

In [1]:
#include<run_hpx.cpp>
#include<vector>
#include<fstream>
#include <iostream>

IncrementalExecutor::executeFunction: symbol '__emutls_v._ZSt11__once_call' unresolved while linking function '__cxx_global_var_initcling_module_1_.28'!
IncrementalExecutor::executeFunction: symbol '__emutls_v._ZSt15__once_callable' unresolved while linking function '__cxx_global_var_initcling_module_1_.28'!


## New class to manage the partitions

In [2]:
class partition_data
{
public:
    partition_data(std::size_t size = 0)
      : data_(size)
    {}

    partition_data(std::size_t size, double initial_value)
      : data_(size)
    {
        double base_value = double(initial_value * size);
        for (std::size_t i = 0; i != size; ++i)
            data_[i] = base_value + double(i);
    }

    double& operator[](std::size_t idx) { return data_[idx]; }
    double operator[](std::size_t idx) const { return data_[idx]; }

    std::size_t size() const { return data_.size(); }

private:
    std::vector<double> data_;
};


## Method to save the out to a file for ploting the result

In [3]:
void saveFile(std::vector<partition_data> U){

  std::ofstream myfile;
  myfile.open("out.txt");
  
  for( auto u : U)
      for (size_t i = 0 ; i < u.size(); i++)
          myfile <<  u[i] << std::endl;
  myfile.close();

}

## Default parameters

In [4]:
double k = 0.5;     // heat transfer coefficient
double dt = 1.;     // time step
double dx = 1.;     // grid spacing
size_t nx = 100;
size_t nt = 45;
size_t np = 1;

## Function to map the indicies 

In [5]:
inline std::size_t idx(std::size_t i, int dir, std::size_t size)
{
    if(i == 0 && dir == -1)
        return size-1;
    if(i == size-1 && dir == +1)
        return 0;

    HPX_ASSERT((i + dir) < size);

    return i + dir;
}

## Simulation class

In [6]:
class stepper
{

public:

    // Our data for one time step
    typedef partition_data partition;
    typedef std::vector<partition> space;

    // Our operator
    static double heat(double left, double middle, double right)
    {
        return middle + (k*dt/(dx*dx)) * (left - 2*middle + right);
    }

    // The partitioned operator, it invokes the heat operator above on all
    // elements of a partition.
    static partition_data heat_part(partition_data const& left,
        partition_data const& middle, partition_data const& right)
    {
        std::size_t size = middle.size();
        partition_data next(size);

        next[0] = heat(left[size-1], middle[0], middle[1]);

        for (std::size_t i = 1; i != size-1; ++i)
            next[i] = heat(middle[i-1], middle[i], middle[i+1]);

        next[size-1] = heat(middle[size-2], middle[size-1], right[0]);

        return next;
    }

    // do all the work on 'np' partitions, 'nx' data points each, for 'nt'
    // time steps
    space do_work(std::size_t np, std::size_t nx, std::size_t nt)
    {
        // U[t][i] is the state of position i at time t.
        std::vector<space> U(2);
        for (space& s: U)
            s.resize(np);

        // Initial conditions: f(0, i) = i
        for (std::size_t i = 0; i != np; ++i)
            U[0][i] = partition_data(nx, double(i));

        // Actual time step loop
        for (std::size_t t = 0; t != nt; ++t)
        {
            space const& current = U[t % 2];
            space& next = U[(t + 1) % 2];

            for (std::size_t i = 0; i != np; ++i)
                next[i] =
                heat_part(current[idx(i, -1, np)], current[i], current[idx(i, +1, np)]);
        }

        // Return the solution at time-step 'nt'.
        return U[nt % 2];
    }
};

## Start simulation

In [7]:
stepper step;

In [8]:
run_hpx([](){


std::uint64_t t = hpx::util::high_resolution_clock::now();

stepper::space solution = step.do_work(np, nx, nt);


std::uint64_t elapsed = hpx::util::high_resolution_clock::now() - t;
std::uint64_t const os_thread_count = hpx::get_os_thread_count();

std::cout << "Computation took " << t << " on " << os_thread_count << " threads" << std::endl;
    
std::cout << "Output written to out.txt!" << std::endl;
saveFile(solution);

});

Computation took 9090768765537605 on 4 threads
Output written to out.txt!


(void) @0x7fdadf192be8
